In [61]:
import numpy as np
import pandas as pd

from pandas.io.json import json_normalize
import functools
import json
import ast

from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
# Dataset paths
business_path = 'data/yelp_academic_dataset_business.json'
checkin_path = 'data/yelp_academic_dataset_checkin.json'
review_path = 'data/yelp_academic_dataset_review.json'
tip_path = 'data/yelp_academic_dataset_tip.json'
user_path = 'data/yelp_academic_dataset_user.json'

## Business data

За да добиеме слика околу тоа како изгледа податочното множество подолу се дадени атрибутите и нивното значење.

**business_id** - уникатен идентификатор за бизнисот составен од 22 карактери <br>
**name** - име на бизнисот <br>
**address** - адреса на бизнисот <br>
**city** - град <br>
**state** - код на state-от, 2 карактери <br>
**postal_code** - поштенски број <br>
**latitude**, **longitude** - координати на бизнисот <br>
**stars** - оценка заокружена на половина ѕвезда <br>
**review_count** - број на review-а <br>
**is_open** - дали бизнисот е отворен или затворен. Вредностите се 0 и 1 за отворен или затворен соодветно <br>
**attributes** - атрибути за биснисот. Вредноста е објект составен од име на атрибутот и вредност која може да биде проста или објект <br>
**categories** - листа на категории <br>
**hours** - објект кој мапира ден во неделата со работно време во 24 часовен формат <br>


In [3]:
business = pd.read_json(business_path, lines=True)

In [4]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,1,None,"Pets, Pet Services, Pet Groomers",None
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Hardware Stores, Home Services, Building Suppl...","{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ..."
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."


Пред да поминеме на чистење на потоците би требале да се справиме со атритубите `attributes` и `hour` кои претставуваат објекти, но и со `categories` што содржи листа од вредности. Соодветно, за полесно справување со овие атрибути истите ќе ги претавиме како посебни колони зависно од нивната вредсност.

Бидејќи ваквата операција ќе генерира голем број на вредности коишто недостасуваат, добро е да имаме увид во бројот на вредности кои недостасуваат пред "нормализација" на json вредностите:

In [182]:
print(business.isna().sum().to_string())

business_id         0
name                0
address             0
city                0
state               0
postal_code         0
latitude            0
longitude           0
stars               0
review_count        0
is_open             0
attributes      29045
categories        524
hours           44843


**attributes** - атрибути за биснисот. Вредноста е објект составен од име на атрибутот и вредност која може да биде проста или објект <br>

In [6]:
def is_json(value):
    try:
        json_object = ast.literal_eval(value)
    except ValueError as e:
        return False
    
    if isinstance(json_object, dict):
        return True
    
    return False

def attributes_mapper(value):
    if value is None:
        return {}
    else:
        return {k: ast.literal_eval(v) if is_json(v) else v for k, v in value.items()}
            

In [7]:
attribute_values = business.attributes.apply(attributes_mapper).tolist()
normalized_attributes = json_normalize(attribute_values, sep='_')
# business = pd.concat([business, normalized_attributes], axis=1)
# business = business.drop('attributes', axis=1)
# unique_attributes = functools.reduce(lambda x, y: x.union(list(y.keys())), attribute_values, set())

**hours** - објект кој мапира ден во неделата со работно време во 24 часовен формат <br>

Тука треба да направиме разлика помеѓу оние коишто немаат поставено работно време и оние коишто не работат во одредени денови. Соодветно, бизнисите коишто не работат во одредени денови за тој ден ќе имаат вредност *`'Closed'`*.

In [8]:
functools.reduce(lambda x, y: x.union(list(y.keys())) if y is not None else x.add('None') or x, 
                 business.hours.tolist(), set())

{'Friday',
 'Monday',
 'None',
 'Saturday',
 'Sunday',
 'Thursday',
 'Tuesday',
 'Wednesday'}

In [9]:
def hours_mapper(hours):
    if hours is None:
        return {}
    
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    return {day: 'Closed' if day not in hours.keys() else hours[day] for day in days}

In [10]:
hours_values = business.hours.apply(hours_mapper).tolist()
normalized_hours = json_normalize(hours_values, sep='_')

In [11]:
normalized_hours

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,10:0-18:0,11:0-20:0,10:0-18:0,11:0-20:0,11:0-20:0,11:0-20:0,13:0-18:0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7:0-16:0,7:0-16:0,7:0-16:0,7:0-16:0,7:0-16:0,Closed,Closed
4,0:0-0:0,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,Closed,Closed
...,...,...,...,...,...,...,...
209388,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,Closed
209389,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209390,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-19:0
209391,0:0-0:0,7:0-19:0,7:0-19:0,7:0-19:0,7:0-19:0,9:0-19:0,9:0-19:0


**categories** - листа на категории <br>

Оваа колона ќе ја поделам на онолку колку колку што категории постојат. Вредноста за соодветната инстанца ќе биде `1` или `0` во зависност од тоа дали бизнисот припаѓа или не на категоријата соодветно. Со инстанците коишто немаат асоцирана вредност ќе се справиме подоцна. 

In [157]:
def categories_mapper(categories):
    if categories is None:
        return []
    
    name_filter = lambda x: x if x.isalpha() else False
    categories_list = [''.join(filter(name_filter, category)) for category in categories.split(', ')]
    
    return categories_list

In [247]:
categories_values = business.categories.apply(categories_mapper)
business_categories = functools.reduce(lambda x, y: x.union(categories_mapper(y)) or x, 
                                       business.categories, set())

In [244]:
mlb = MultiLabelBinarizer(sparse_output=True)
categories_matrix = mlb.fit_transform(categories_values)
categories_matrix = categories_matrix.astype('float64')
categories_matrix[business[business['categories'].isnull()].index, :] = np.nan
normalized_categories = pd.DataFrame.sparse.from_spmatrix(categories_matrix, columns=mlb.classes_)

/Users/darko/anaconda3/envs/nlp_finki/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [258]:
print('Number of categories: ', len(business_categories))

Number of categories:  1336


In [261]:
business = pd.concat([business, normalized_attributes, normalized_categories, normalized_hours], axis=1)
business = business.drop(['attributes', 'categories', 'hours'], axis=1)

Моментална состојба на вредностите коишто недостасуваат.

In [262]:
print(business.isna().sum().to_string())

business_id                               0.0
name                                      0.0
address                                   0.0
city                                      0.0
state                                     0.0
postal_code                               0.0
latitude                                  0.0
longitude                                 0.0
stars                                     0.0
review_count                              0.0
is_open                                   0.0
BusinessAcceptsCreditCards            87156.0
BikeParking                          119628.0
GoodForKids                          140858.0
ByAppointmentOnly                    148594.0
RestaurantsPriceRange2                98105.0
BusinessParking_garage                97790.0
BusinessParking_street                97979.0
BusinessParking_validated             97790.0
BusinessParking_lot                   97864.0
BusinessParking_valet                 97190.0
DogsAllowed                       

Сега имаме множество коешто е во форма на матрица и е поразбирливо за моделите од машинско учење. Сепак, за да ги подобриме нашите резултати треба најпрвин да се справиме со вредностите коишто недостасуваат, а потоа и да ги конвертираме податоците за нивно полесна обработка.